텐서플로는 텐서를 흘려보내면서 데이터를 처리하는 라이브러리
텐서는 다차원 배열을 의미
랭크는 텐서의 차원을 의미. 만약 랭크가 0이면 스칼라 값, 랭크가 1이면 벡터값, 랭크가 2라면 행렬, 3이상이면 텐서라고 부른다.
텐서들을 계산 그래프 구조 (Computational Graph)를 통해서 노드에서 노드로 이동

텐서플로 작성시에는 다음의 과정을 거쳐야 한다.
1. 그래프 생성
2. 그래프 실행

그래프 생성과 실행을 분리하는 이유는 실행과정에서 전체 그래프 중 일부만 사용하게 될 수도 있기 때문이다.

노드에 연산, 변수, 상수 등을 정의하고 노드 간의 연결인 엣지를 통해 텐서를 주고받으면서 계산을 수행한다.


In [1]:
# 텐서플로 라이브러리 임포트
import tensorflow as tf

# 상수값을 표현하는 tf.constant 노드 2개 정의하고 print 함수로 출력
node1 = tf.constant(3.0, dtype = tf.float32)
node2 = tf.constant(4.0) # 암시적으로 tf.float32 타입으로 선언됨
print(node1, node2)


Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


위에서 상수 선언은 그래프의 노드 형태를 정의한 것일 뿐 실제로 값을 할당한 것은 아니다.
3과 4라는 값을 출력하려면 정의한 그래프를 실행해야 한다. 그래프를 실행하지 않았으므로 노드의 정보만을 출력한다.

텐서플로에서 그래프의 실행은 세션을 열어서 수행한다.



In [2]:
# 세션을 열고 그래프를 실행
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [3]:
# 2개의 노드의 값을 더하는 연산을 수행하는 node3을 정의한다.
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ", sess.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3):  7.0


In [4]:
# 모든 코드를 실행했으니 세션을 닫는다.
sess.close()

# 플레이스홀더

임의의 값을 입력받아서 임의의 값을 출력할 수 있도록 하기 위해 플레이스홀더와 변수를 사용한다.
플레이스홀더 노드는 그래프에서 임의의 입력값을 받을 수 있도록 한다.
`tf.placeholder` 는 `feed_dict`를 통해 입의의 갑을 입력받을 수 있는 연산이다.


In [5]:
# 텐서플로 라이브러리 임포트
import tensorflow as tf

# 플레이스홀더 노드와 add 노드를 유지
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # 암시적으로 tf.add(a, b) 형태로 정의됨

# 세션을 열고 플레이스홀더에 값을 넣고, 그래프를 실행시켜 실제 연산을 수행
# sess.run의 1번째 인자로 실행하고자 하는 연산을 명시
# 2번째 인자로 실제로 넣을 값들을 Dictionary 형태로 명시하는 feed_dict를 선언하고, feed_dict 부분에 플레이스홀더 노드에 넣을 값을 지정
sess = tf.Session()
print(sess.run(adder_node, feed_dict = {a: 3, b: 4.5}))
print(sess.run(adder_node, feed_dict = {a: [1, 3], b: [2, 4]}))

# 노드를 추가해서 더 복잡한 그래프 형태를 만든다.
add_and_triple = adder_node * 3 # 암시적으로 tf.multiply(adder_node, 3) 형태로 정의됨
print(sess.run(add_and_triple, feed_dict = {a: 3, b: 4.5}))

sess.close()


7.5
[3. 7.]
22.5


# 선형회귀 및 경사하강법 알고리즘

## 머신러닝의 기본 프로세스 - 가설 정의, 손실 함수 정의, 최적화 정의

회귀란 어떤 실수 값을 예측하는 문제
예측하는 값이 실수가 아니라 이산값이라면 분류라고 부름

모든 머신러닝 모델은 다음의 3가지 과정을 거친다.
1. 학습하고자 하는 가설 h(theta)을 수학적 표현식으로 나타낸다.
텐서플로에서는 변수 노드를 선언해서 파라미터로 사용하고, 플레이스홀더 노드를 선언해서 인풋데이터, 타겟 데이터를 받는다.
적절한 파라미터 갑을 알아내기 위해서는 현재 파라미터값이 우리가 풀고자 하는 목적에 적합한 값인지를 측정할 수 있어야 한다.


2. 가설의 성능을 측정할 수 있는 손실함수 J(theta)을 정의한다.
손실함수로는 평균제곱오차 (MSE)가 가장 자주 쓰인다.
MSE는 모델의 예측값과 실제 타겟값과의 차이를 제곱해서 모두 더한 후, 평균을 취하는 함수이다.
따라서 예측값이 실제 타겟값과 가까울수록 작은 값을 갖는다.

손실함수는 우리가 풀고자 하는 목적에 가까운 형태로 파라미터가 최적화 되었을 때, 더 작은 값을 갖는 특성을 가져야 한다.
이런 특징 때문에 비용함수라고도 부른다.


3. 손실함수를 최소화할 수 있는 학습 알고리즘을 설계한다.
파라미터를 적절한 값으로 업데이트하는 알고리즘을 최적화 기법이라고 한다.
가장 대표적인 기법은 경사하강법이다. 경사하강법은 현재 스텝의 파라미터에서 손실 함수의 미분값에 러닝레이트를 곱한만큼 빼서 다음 스텝의 파라미터 값으로 지정한다.
따라서 손실함수의 미분값이 크면 하나의 스텝에서 파라미터가 많이 업데이트되고 작으면 적게 업데이트 된다.
경사하강법은 손실함수가 최소가 되는 지점에서 종료하는 것이 이상적이다. 하지만 언제 최소값이 될지 알 수 없으므로 적당한 횟수만큼 업데이트를 진행한 후에는 학습을 종료한다.



In [9]:
import tensorflow as tf

# 변수와 플레이스홀더를 이용해서 선혀회귀모델의 그래프 구조 정의
W = tf.Variable(tf.random_normal(shape = [1]), name = "W")
b = tf.Variable(tf.random_normal(shape = [1]), name = "b")
x = tf.placeholder(tf.float32, name = "x")
linear_model = W * x + b

# True Value를  입력받을 플레이스홀더를 정의
y = tf.placeholder(tf.float32, name = "y")

# 평균제곱오차 손실함수를 정의
loss = tf.reduce_mean(tf.square(linear_model - y))

# 손실함수의 값의 변화를 추적하기 위해 요약정보 로그를 남기는 코드 추가
tf.summary.scalar('loss', loss)

# 최적화를 위한 옵티마이저를 정의
# 옵티마이저로 경사하강법을 사용하고 러닝레이트는 0.01로 설정
optimizer = tf.train.GradientDescentOptimizer(0.01)
train_step = optimizer.minimize(loss)

# 학습을 위한 트레이닝 데이터와 테스트 데이터를 준비
x_train = [1, 2, 3, 4]
y_train = [2, 4, 6, 8]

# 세션을 열고 파라미터 (W, b)를 normal distribution에서 추출한 임의의 값으로 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# tf.summary.merge_all API를 이용하여 모든 요약 정보를 하나로 합치는 연산 정의
merged = tf.summary.merge_all()

# 지정한 경로에 요약 정보를 직렬화해서 파일로 저장하는 tf.summary.FileWriter를 정의
tensorboard_writer = tf.summary.FileWriter('./tensorboard_log', sess.graph)

# 경사하강법을 1000번 수행
for i in range(1000):
    sess.run(train_step, feed_dict = {x: x_train, y: y_train})
    
    # 스텝마다 텐서보드 요약 정보 값들을 계산해서 지정된 경로에 저장
    summary = sess.run(merged, feed_dict = {x: x_train, y: y_train})
    tensorboard_writer.add_summary(summary, i)
    
# 테스트를 위한 입력값을 준비
x_test = [3.5, 5, 5.5, 6]
# 테스트 데이터를 이용해 학습된 선형 회귀 모델이 데이터의 경향성을 잘 학습했는지 측정
print(sess.run(linear_model, feed_dict = {x: x_test}))

sess.close()


InvalidArgumentError: You must feed a value for placeholder tensor 'x' with dtype float
	 [[node x (defined at <ipython-input-8-99584e27f7eb>:6) ]]

Original stack trace for 'x':
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/hyegeuncho/venv/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/hyegeuncho/venv/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/hyegeuncho/venv/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/Users/hyegeuncho/venv/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in 

# 텐서보드를 이용한 그래프 시각화

텐서보드를 사용하는 패턴 2가지

1. tf.summary.scalar API를 이용한 스탭마다 손실함수 출력
2. 계산 그래프 시각화

